In [4]:
import spotipy
import os
from spotipy.oauth2 import SpotifyOAuth
from dotenv import load_dotenv
import pprint

In [5]:
load_dotenv()

client_id = os.getenv("SPOTIFY_CLIENT_ID")
client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")
redirect_uri = os.getenv("SPOTIFY_REDIRECT_URI")
playlist_id = os.getenv("SPOTIFY_PLAYLIST_ID")

scope = "user-library-read"

In [6]:
def get_playlist_tracks(playlist_id):
    names = []
    # Create an OAuth object
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
        client_id=client_id, 
        client_secret=client_secret,
        redirect_uri=redirect_uri,
        scope=scope
    ))

    results = sp.playlist(playlist_id)
    
    tracks = results['tracks']['items']
    
    for item in tracks:
        names.append(item['track']['name'])
    
    return names

In [7]:
# Crack playlist ID: 3xWx987zAhFtL1r5kPhtpr

crack = get_playlist_tracks("3xWx987zAhFtL1r5kPhtpr")
print(crack)

['Rodeo (Remix)', 'One Of The Girls (with JENNIE, Lily Rose Depp)', 'Confident', 'Her Way', 'Belong to the City', 'PERSIAN RUGS', 'FREAK IN YOU', 'Earned It (Fifty Shades Of Grey)', 'Streets', 'Wine Pon You (feat. Konshens)', 'Girls Need Love (with Drake) - Remix', "Don't", 'Wus Good / Curious', 'Coming Down', 'The Party & The After Party', 'What You Need', 'Sativa', 'Feel It', 'Sex With Me', 'Needed Me', 'Hotel', 'Woo', 'Lost in the Fire (feat. The Weeknd)', 'Often', 'Into It', "Sun Down, I'm Up - Sped Up & Pitched", 'Call Out My Name', 'Cherry Hill', 'All The Time', 'Excitement', 'One Of The Girls (with JENNIE, Lily Rose Depp)', 'Wine Pon You (feat. Konshens)', 'Break from Toronto', "Don't Run", 'Dreamin', 'SOMETHING ABOUT YOU', 'TBH', 'GREEDY', 'Come and See Me (feat. Drake)', 'Recognize (feat. Drake)', 'THINGS & SUCH', 'Say It', "Let Em' Know", 'Poison', 'Been Away', 'Forever Yours', 'Chanel', 'R e s e n t m e n t', 'N o C h i l l', 'Stay Ready (What A Life)', 'Grown Woman', 'WEST 

In [8]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

In [ ]:
# Create the flow using the client secrets file from the Google API Console.
flow = InstalledAppFlow.from_client_secrets_file( 
    'client_secret.json',
    scopes=['https://www.googleapis.com/auth/youtube.force-ssl']) # This grants full access to the YouTube account

""" Essentially, flow will get an authentication code and will use that alongside the client_secrets.json file to retrieve an 
access token that allows our app to get what's defined by the scopes"""


credentials = flow.run_local_server(port=8081) # Store the access and refresh tokens in the credentials object

youtube = build('youtube', 'v3', credentials=credentials) # build the YouTube service object; gives access to YouTube API 

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=175375944583-k4t537kob18t0tt5621dbm0rhm8q69u4.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8081%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=7X0LJ8PxhUtKqzUQfTjSIArnxpD4K3&access_type=offline


In [ ]:
def search_song(query):
    result = youtube.search().list(part="snippet", q=query, type="video", maxResults=1).execute()
    # result['items'][0]['snippet']['title']
    return result

# pprint.pprint(search_song("Some of Your Love PND Slowed Reverb"))

In [10]:
# In this cell, we want to call search_song() on all of the songs in our 'crack' playlist

slowed_version = []

for song in crack[:5]: 
    slowed_version.append(search_song(song + " Slowed Reverb"))

pprint.pprint(slowed_version)

HttpError: <HttpError 403 when requesting https://youtube.googleapis.com/youtube/v3/search?part=snippet&q=Rodeo+%28Remix%29+Slowed+Reverb&type=video&maxResults=1&alt=json returned "The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.". Details: "[{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]">

In [ ]:
def create_playlist(title, description=None, privacyStatus='private', defaultLanguage=None):
    snippet_body = {
        'snippet': {
            'title': title,
            'description': description,
            'defaultLanguage': defaultLanguage
        },
        'status': {
            'privacyStatus': privacyStatus
        }
    }
    return youtube.playlists().insert(part="snippet,status", body=snippet_body).execute()

In [ ]:
result = create_playlist(title="Test")
# result['id'] is the playlist ID

In [ ]:
def add_songs(videoID, playlistID):
    snippet_body = {
        "snippet": {
            "playlistID" : playlistID,
            "resourceID": {
                "kind": "youtube#video",
                "videoID": videoID
            }
        }
    }
    # This makes a POST request to the YouTube API
    return youtube.playlistItems().insert(part="snippet", body=snippet_body).execute()